In [ ]:
#Importing Required Models
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import pickle
import cv2
import glob
import os

In [ ]:
car_images = glob.glob('./vehicles/vehicles/**/*.png')
non_car_images = glob.glob('./non-vehicles/non-vehicles/**/*.png')
print('The no of car images is : ' + str(len(car_images)))
print('The no of non-car images is : ' + str(len(non_car_images)))
print('The total size of the data set is : '+ str(len(car_images)+len(non_car_images)))

if(np.absolute(len(car_images)-len(non_car_images))<0.1*(len(car_images)+len(non_car_images))):
    print('The Dataset is balanced')
else:
    print('Data Augmentation is required')

In [ ]:
car_img = cv2.imread(car_images[np.random.randint(0,len(car_images))])
car_img = cv2.cvtColor(car_img,cv2.COLOR_BGR2RGB)
non_car_img = cv2.imread(non_car_images[np.random.randint(0,len(non_car_images))])
non_car_img = cv2.cvtColor(non_car_img,cv2.COLOR_BGR2RGB)
plt.imshow(car_img)
plt.show()
plt.imshow(non_car_img)
plt.show()

In [ ]:
#function to compute binned color features
def bin_spatial(img, size=(32, 32)):
    
    features = cv2.resize(img, size).ravel() 
    return features


In [ ]:
# function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
   
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    
    return hist_features


In [ ]:
def get_hog_features(img, orient=9, pix_per_cell=8, cell_per_block=2, 
                        vis=False, feature_vec=True):
    
    #If vis is True return the Image to display it
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        
        return features, hog_image
    
    
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        
        return features

In [ ]:
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256),orient=9,pix_per_cell=8, cell_per_block=2, hog_channel=0):
   
    features = []
    
    for file in imgs:
        
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: 
            feature_image = np.copy(image)      
        
        # Applying bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        
        # Applying color_hist() to get historgram features
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        
        #Hog features extraction
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)

        
        features.append(np.concatenate((spatial_features, hist_features,hog_features)))
            

    return features

In [ ]:
#Visulazing the hog image on car and non-car data
car_hog_features,car_hog_image=get_hog_features(car_img[:,:,2], vis=True, feature_vec=True)
non_car_hog_features,non_car_hog_image=get_hog_features(non_car_img[:,:,2], vis=True, feature_vec=True)

# Visualize 
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(7,7))
f.subplots_adjust(hspace = .4, wspace=.2)
ax1.imshow(car_img)
ax1.set_title('Car Image', fontsize=16)
ax2.imshow(car_hog_image, cmap='gray')
ax2.set_title('Car HOG', fontsize=16)
ax3.imshow(non_car_img)
ax3.set_title('Non-Car Image', fontsize=16)
ax4.imshow(non_car_hog_image, cmap='gray')
ax4.set_title('Non-Car HOG', fontsize=16)
print('...')

In [ ]:
#HyperParameters for feature Extraction
colorspace='YCrCb'
spatial_size=(32, 32)
hist_bins=32
hist_range=(0, 256)
orient=9
pix_per_cell=8
cell_per_block=2
hog_channel=0


car_features = extract_features(car_images, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel,spatial_size=(32, 32),hist_bins=32, hist_range=(0, 256))
not_car_features = extract_features(non_car_images, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel,spatial_size=(32, 32),hist_bins=32, hist_range=(0, 256))

In [ ]:
tot_features=np.vstack((car_features, not_car_features)).astype(np.float64)
tot_labels=np.hstack((np.ones(len(car_features)), np.zeros(len(not_car_features))))

print(tot_features.shape)
print(tot_labels.shape)

In [ ]:
#Splitting data into random train and test sets
#Randstate is any number in 0 to 99
rand_state = np.random.randint(0, 100)
X_train,X_test,y_train,y_test=train_test_split(tot_features,tot_labels,test_size=0.2,random_state=rand_state)

In [ ]:
# Fit a per-column scaler
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

import time
import pickle
svc = LinearSVC()

start=time.time()
print('Model Training Started')
svc.fit(X_train,y_train)
print('The testing acuuracy is: '+ str(round(svc.score(X_test,y_test),4)))
end=time.time()

print('The Time taken for model training and predcition is: '+str(end-start)+ ' s')

pickle.dump(svc,open('svc.sav','wb'))
svc=pickle.load(open('svc.sav', 'rb'))
print('Model is loaded')

In [ ]:
#Classroom code

# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins,hog_channel,cspace):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    rectangles=[]
    
    img_tosearch = img[ystart:ystop,:,:]
    
    if cspace != 'RGB':
        if cspace == 'HSV':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HSV)
        elif cspace == 'LUV':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2LUV)
        elif cspace == 'HLS':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HLS)
        elif cspace == 'YUV':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YUV)
        elif cspace == 'YCrCb':
            ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YCrCb)
    else: 
        ctrans_tosearch = np.copy(img_tosearch) 
    
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
    
    if(hog_channel=='ALL'):
        ch1 = ctrans_tosearch[:,:,0]
        ch2 = ctrans_tosearch[:,:,1]
        ch3 = ctrans_tosearch[:,:,2]
    else:
        ch=ctrans_tosearch[:,:,hog_channel]

    # Define blocks and steps as above
    nxblocks = (ctrans_tosearch.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ctrans_tosearch.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    # Compute individual channel HOG features for the entire image
    if(hog_channel=='ALL'):
        hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    else:
        hog=get_hog_features(ch, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            if(hog_channel=='ALL'):
                hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
            else:
                hog_features=hog[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                rectangles.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))
                
    return rectangles

In [ ]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:

        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

In [ ]:
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

In [ ]:
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    rects = []
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        rects.append(bbox)
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image and final rectangles
    return img, rects

In [ ]:
#Code taken from github https://github.com/jeremy-shannon/CarND-Vehicle-Detection/blob/master/vehicle_detection_project.ipynb


def image_pipeline(image):
    
    colorspace='YCrCb'
    spatial_size=(32, 32)
    hist_bins=32
    hist_range=(0, 256)
    orient=9
    pix_per_cell=8
    cell_per_block=2
    hog_channel=0

    rectangles = []

    ystart = 400
    ystop = 464
    scale = 1.0
    rectangles.append(find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, 
                                cell_per_block, spatial_size, hist_bins,hog_channel,colorspace))
    ystart = 416
    ystop = 480
    scale = 1.0
    rectangles.append(find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, 
                                cell_per_block, spatial_size, hist_bins,hog_channel,colorspace))
    ystart = 400
    ystop = 496
    scale = 1.5
    rectangles.append(find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, 
                                cell_per_block, spatial_size, hist_bins,hog_channel,colorspace))
    ystart = 432
    ystop = 528
    scale = 1.5
    rectangles.append(find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, 
                                cell_per_block, spatial_size, hist_bins,hog_channel,colorspace))
    ystart = 400
    ystop = 528
    scale = 2.0
    rectangles.append(find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, 
                                cell_per_block, spatial_size, hist_bins,hog_channel,colorspace))
    ystart = 432
    ystop = 560
    scale = 2.0
    rectangles.append(find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, 
                                cell_per_block, spatial_size, hist_bins,hog_channel,colorspace))
    ystart = 400
    ystop = 596
    scale = 3.5
    rectangles.append(find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, 
                                cell_per_block, spatial_size, hist_bins,hog_channel,colorspace))
    ystart = 464
    ystop = 660
    scale = 3.5
    rectangles.append(find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, 
                                cell_per_block, spatial_size, hist_bins,hog_channel,colorspace))
    
    rectangles = [item for sublist in rectangles for item in sublist] 
    
    heatmap_img = np.zeros_like(image[:,:,0])
    heatmap_img = add_heat(heatmap_img, rectangles)
    heatmap_img = apply_threshold(heatmap_img, 1)
    labels = label(heatmap_img)
    draw_img, rects = draw_labeled_bboxes(np.copy(image), labels)
    
    
    return draw_img

In [ ]:
test_img = mpimg.imread('./test_images/test1.jpg')
out_img = image_pipeline(test_img)
plt.imshow(out_img)

In [ ]:
images=os.listdir("test_images/")
for i in images:
    image = mpimg.imread('test_images/'+str(i))
    result = image_pipeline(image)
    mpimg.imsave("output_images_hog/"+str(i), result)

In [ ]:
#Exeuting the video and saving it to the output folder
yellow_output = 'project_video_output_HOG.mp4'
clip2 = VideoFileClip('test_video.mp4')
yellow_clip = clip2.fl_image(image_pipeline)
%time yellow_clip.write_videofile(yellow_output, audio=False)